<a href="https://colab.research.google.com/github/roxcalan/gen-ai/blob/main/gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

In [2]:
! pip install git+https://github.com/huggingface/transformers -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 5.6 MB/s eta 0:00:00


In [3]:
from transformers import pipeline

## ***Audio to Text Generation***

In [4]:
whisper = pipeline('automatic-speech-recognition', model = 'openai/whisper-medium')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [5]:
text = whisper('/content/tp1.opus')

FileNotFoundError: [Errno 2] No such file or directory: '/content/tp1.opus'

In [ ]:
text

## ***Text to Sentiment Analysis***

In [ ]:
emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')

In [ ]:
emotion_labels = emotion("I want to die. I am fed up of depression")

In [ ]:
emotion_labels

# _**Text to Image**_/_**Text to Video**_

In [ ]:
# Install required packages
!pip install diffusers accelerate mediapy transformers better_profanity modelscope==1.4.2 open_clip_torch pytorch-lightning
from diffusers import EulerAncestralDiscreteScheduler as EAD
from diffusers import StableDiffusionPipeline as sdp
import mediapy as mp
from modelscope.pipelines import pipeline
from modelscope.outputs import OutputKeys
import torch
from google.colab import files
from better_profanity import Profanity

In [ ]:
# Load text-to-image synthesis model
import matplotlib.pyplot as plt
def generate_image(prompt):
    model = "dreamlike-art/dreamlike-photoreal-2.0"
    scheduler = EAD.from_pretrained(model, subfolder="scheduler")

    if model.startswith("stabilityai/"):
        model_revision = "fp16"
    else:
        model_revision = None

    device = "cuda" if torch.cuda.is_available() else "cpu"

    pipe = sdp.from_pretrained(
        model,
        scheduler=scheduler,
        torch_dtype=torch.float16,
        revision=model_revision,
    )
    pipe = pipe.to(device)

    def has_profanity(text):
        return Profanity().contains_profanity(text)

    def filter_text(text):
        while has_profanity(text):
            text = input("Please provide an alternative prompt: ")
        return text

    filtered_input = filter_text(prompt)

    images = pipe(
        filtered_input,
        height=512,
        width=512,
        num_inference_steps=30,
        guidance_scale=12,
        num_images_per_prompt=1
    ).images

    return images[0]


# Load text-to-video synthesis model
def generate_video(prompt):
    p = pipeline('text-to-video-synthesis', 'damo/text-to-video-synthesis')
    test_text = {'text': prompt}
    output_video_path = p(test_text)[OutputKeys.OUTPUT_VIDEO]
    return output_video_path


def main():
    prompt = input("Enter your prompt: ")
    output_type = input("Enter 'image' or 'video' to generate: ")

    if output_type.lower() == 'image':
        image = generate_image(prompt)
        # Display the image using Matplotlib
        plt.imshow(image)
        plt.axis('off')
        plt.show()

    elif output_type.lower() == 'video':
        video_path = generate_video(prompt)
        print('Video generated:', video_path)
        files.download(video_path) #Once the video is generated, it gets downloaded in the downloads folder of the computer. You can view it from there.
    else:
        print("Invalid output type. Please enter 'image' or 'video'.")



# Run the main function
if __name__ == "__main__":
    main()


## _**HUG CHAT**_

In [ ]:
!pip install hugchat

from hugchat import hugchat
from hugchat.login import Login

sign = Login("Mubeen161", "Mubeen161#") #Please enter your own Hugging face (website) credentials ie, Login ID and Password.
cookies = sign.login()

cookie_path_dir = "./cookies_snapshot"
sign.saveCookiesToDir(cookie_path_dir)

chatbot = hugchat.ChatBot(cookies=cookies.get_dict())

id = chatbot.new_conversation()
chatbot.change_conversation(id)
msg= input("Enter your prompt")

print(chatbot.chat(msg))